# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from nltk.tokenize import sent_tokenize
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [2]:
dvach = open('2ch_corpus.txt').read()
news = open('lenta.txt').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [4]:
from collections import Counter

In [5]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [6]:
vocab_dvach.most_common(10)

[('и', 55892),
 ('в', 48853),
 ('не', 46602),
 ('на', 29660),
 ('что', 26668),
 ('я', 21734),
 ('а', 21310),
 ('с', 21080),
 ('это', 17727),
 ('ты', 15469)]

In [7]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

In [8]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(10)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357),
 ('я', 0.011691602907246653),
 ('а', 0.011463516055646737),
 ('с', 0.011339789697467536),
 ('это', 0.009536074571489897),
 ('ты', 0.008321404498582796)]

In [9]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(10)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205)]

In [10]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
sentences_dvach = sent_tokenize(dvach)
sentences_news = sent_tokenize(news)

In [12]:
test_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][-50:]
test_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][-50:]

In [13]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][:-50]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][:-50]

In [14]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [15]:
trigrams_dvach.most_common(10)

[('<start> <start> а', 7826),
 ('<start> <start> я', 5949),
 ('<start> <start> и', 4120),
 ('<start> <start> в', 4047),
 ('<start> <start> ну', 3927),
 ('<start> <start> но', 3273),
 ('<start> <start> это', 3041),
 ('<start> <start> не', 2968),
 ('<start> <start> ты', 2925),
 ('<start> <start> да', 2596)]

In [16]:
trigrams_news.most_common(10)

[('<start> <start> в', 7961),
 ('<start> <start> по', 6210),
 ('<start> <start> как', 3736),
 ('<start> <start> однако', 1693),
 ('<start> <start> на', 1642),
 ('<start> <start> об', 1618),
 ('<start> об этом', 1578),
 ('<start> <start> он', 1551),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324)]

In [17]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))
id2unigram_dvach = list(unigrams_dvach)
word2id_dvach_uni = {word:i for i, word in enumerate(id2unigram_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach_uni[word3]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])

In [18]:
matrix_dvach = csr_matrix(matrix_dvach)

In [19]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))
id2unigram_news = list(unigrams_news)
word2id_news_uni = {word:i for i, word in enumerate(id2unigram_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news_uni[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [20]:
matrix_news = csr_matrix(matrix_news)

In [21]:
def generate(matrix, id2bigram, id2unigram, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text.append(id2unigram[chosen])
    
        if id2unigram[chosen] == '<end>':
            chosen = bigram2id['<start> <start>']
        else:
            bigram = id2bigram[current_idx]
            trigram = bigram + ' ' + id2unigram[chosen]
            w1, w2, w3 = trigram.split()
            new_bigram = w2 + ' ' + w3
            chosen = bigram2id[new_bigram]
            
        current_idx = chosen
    
    return ' '.join(text)

In [37]:
print(generate(matrix_dvach, id2bigram_dvach, id2unigram_dvach, bigram2id_dvach).replace('<end>', '\n\n'))

двачую этого всё как полагается резко сильно чтобы критическое мышление 

 мама торт обещала постряпать 

 а вики ещё говорит что в финансах нихуя не делая ничего кроме геймплея я там очень много денег и точная цель зачем ты так говоришь как будто сошедшего сто страниц постеров о войне во вьетнаме-день добрый господа американцы мент-1 явился даже то что она говорит об этом и проблема не в тюрьме но потом я снял свой палец со спускового крючка — всё было по-другому я был на тот свет 

 ты постоянно интересуешься как живут другие и уровень жизни повыше 

 да было


In [23]:
print(generate(matrix_news, id2bigram_news, id2unigram_news, bigram2id_news).replace('<end>', '\n\n'))

как ожидается войдут послы португалии и франции относительно окончательного урегулирования конфликта с городскими властями борьба с международным участием в выборах 26 марта может означать что не имеет 

 в результате беспорядочного и непропорционального использования силы российскими военными любой системы про которую американские военные власти в 1982 году пришли военные 

 это второе подобное предупреждение со стороны старопромысловского старосунженского и ханкалинского районов 

 летчику удалось катапультироваться и остаться в живых три члена экипажа 

 с их проблемами 

 свинина и говядина стоят уже не первый столичный руководитель силовик отправленный в отставку 

 раненный в голову женщина скончалась от множества ножевых ранений


In [24]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [25]:
perplexities = []
for sentence in test_news[:10]:
    prob = []
    
    for ngram in ngrammer(sentence, n=3):
        word1, word2, word3 = ngram.split()

        gram2 = word1 + ' ' + word2
        if gram2 in bigrams_news and ngram in trigrams_news:
            prob.append(np.log(trigrams_news[ngram] / bigrams_news[gram2]))
        else:
            prob.append(np.log(0.00001))
    
    perplexities.append(perplexity(prob))

In [26]:
perplexities

[100000.00000000001,
 41704.89804250306,
 94732.29898771658,
 100000.00000000012,
 38799.77480772991,
 12500.001493332888,
 44527.443106809595,
 29174.044201784793,
 100000.00000000001,
 100000.0000000001]

In [27]:
np.mean(perplexities)

66143.84606398772

In [28]:
perplexities = []
for sentence in test_dvach[:10]:
    prob = []
    
    for ngram in ngrammer(sentence, n=3):
        word1, word2, word3 = ngram.split()

        gram2 = word1 + ' ' + word2
        if gram2 in bigrams_dvach and ngram in trigrams_dvach:
            prob.append(np.log(trigrams_dvach[ngram] / bigrams_dvach[gram2]))
        else:
            prob.append(np.log(0.00001))
    
    perplexities.append(perplexity(prob))

In [29]:
perplexities

[70247.22942196298,
 1453.312720712195,
 77748.62515781814,
 100000.00000000012,
 46104.39983753712,
 7650.66734746975,
 18025.826375016793,
 2746.3634966563855,
 110.34264814658026,
 8230.617749535151]

In [30]:
np.mean(perplexities)

33231.738475485516

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2 других способа определить вероятность несловарных слов: 

1)	решить эту проблему так же, как и с фиксированным словарем:
a)	выбрать заранее заданный словарь (список слов);
b)	на этапе нормализации текста в обучающем сете преобразовать любое слово, не входящее в эту выборку (т.е. несловарное слово), в токен несловарного слова (<UNK>);
c)	оценить вероятность <UNK> по тому, сколько раз оно встречается в тексте (как и вероятность любого другого обычного слова).

2)	Если такого заранее заданного словаря нет, то можно его создать, заменив слова в зависимости от их частотности на <UNK>. Например, можно заменить на <UNK> все слова, которые встречаются менее n раз, где n – это какое-то небольшое число. Или заранее выбрать размер словаря и выбрать топ слов по частотности и заменить остальные на <UNK>. 


2. Что такое сглаживание (smoothing)?

Cглаживание – это такой процесс перераспределения вероятностей. Существуют слова, которые есть в словаре (то есть это не несловарные слова), но появляются в тестовом сете в неизвестном контексте (например, появляются после слова, после которого никогда не появлялись в тренировочном сете). Чтобы языковая модель не приписывала нулевую вероятность таким «невидимым» словам, надо немного снизить вероятность наиболее частотных слов и «отдать» ее таким словам. 